In [4]:
from ultralytics import YOLO

In [2]:
#Training the model for the first time

# Load a model
model = YOLO('yolov8n-seg.pt')  # load a pretrained model (recommended for training)

# Display model information (optional)
model.info()

# Train the model
results = model.train(data='coco8-seg.yaml', epochs=100, imgsz=640)


NameError: name 'YOLO' is not defined

In [5]:
# Opening a model that was alraedy trained & fine-tuned

model = YOLO('runs/segment/train12/weights/best.pt')

In [6]:
results = model('/Users/vihaan/Workspace/Local_AI_Earth/YOLO_Experiments/datasets/coco8-seg/images/train/000000000009.jpg',save=True)  # or img.jpg


image 1/1 /Users/vihaan/Workspace/Local_AI_Earth/YOLO_Experiments/datasets/coco8-seg/images/train/000000000009.jpg: 480x640 6 bowls, 2 broccolis, 83.0ms
Speed: 1.6ms preprocess, 83.0ms inference, 10.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/segment/predict


In [6]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes', 'masks']
masks: ultralytics.engine.results.Masks object
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donu

In [18]:
print(results[0].boxes.data.shape)

torch.Size([8, 6])


: 

In [ ]:
import numpy as np
import cv2

def overlay_predictions(image, boxes, masks, labels=None, colors=None):
    """
    Overlay bounding boxes and masks on the original image.
    
    Parameters:
    - image (numpy.ndarray): Original image of shape (height, width, 3)
    - boxes (list of tuples): List of bounding boxes. Each tuple has 4 values (x1, y1, x2, y2)
    - masks (numpy.ndarray): Array of masks of shape (num_instances, height, width). 
                             Masks should be boolean (True for object pixel, False for background)
    - labels (list of str, optional): List of labels for each instance
    - colors (list of tuples, optional): List of colors for each instance. Each tuple has 3 values (B, G, R).
    
    Returns:
    - overlayed_image (numpy.ndarray): Image with overlaid predictions.
    """
    
    overlayed_image = image.copy()
    
    if colors is None:
        # Generate random colors if not provided
        colors = [(np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256)) for _ in range(len(boxes))]

    for index, box in enumerate(boxes):
        color = colors[index]
        
        # Draw bounding box
        cv2.rectangle(overlayed_image, (box[0], box[1]), (box[2], box[3]), color, 2)
        
        # Draw mask
        mask = masks[index]
        overlayed_image[mask] = 0.5 * overlayed_image[mask] + 0.5 * np.array(color, dtype=np.uint8)
        
        # Draw label if provided
        if labels:
            label = labels[index]
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            thickness = 1
            text_size = cv2.getTextSize(label, font, font_scale, thickness)[0]
            text_origin = (box[0], box[1] - 10)
            cv2.rectangle(overlayed_image, (text_origin[0] - 5, text_origin[1] - text_size[1] - 5),
                          (text_origin[0] + text_size[0] + 5, text_origin[1] + text_size[1] - 5), color, -1)
            cv2.putText(overlayed_image, label, text_origin, font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
    
    return overlayed_image

# Example Usage:
# image = cv2.imread("path_to_image.jpg")
# boxes = [(50, 50, 150, 150), (200, 200, 350, 350)]
# masks = np.zeros((2, image.shape[0], image.shape[1]), dtype=bool)
# masks[0, 60:140, 60:140] = True
# masks[1, 210:330, 210:330] = True
# labels = ["Class1", "Class2"]
# overlayed = overlay_predictions(image, boxes, masks, labels)
# cv2.imshow("Overlayed Image", overlayed)
# cv2.waitKey(0)
# cv2.destroyAllWindows()